# PART 1
**!!! Ne pas relancer cette partie du code, utiliser le csv "new_train_labels.csv" (PART 2) !!!**

import pandas as pd
import numpy as np

# Load the train_label
CSV_PATH = './IPEO_Planet_project/train_labels.csv'
df = pd.read_csv(CSV_PATH, sep = ';')
df.fillna(value=0, inplace = True)
df.head()

# Create DataFrame containing the labels only
df_drop_train = df.drop('image_name', axis=1)

# Get all the unique labels present in dataset
df_concat = pd.concat([df_drop_train[col] for col in df_drop_train])
label = df_concat.unique()

# Add the label 'image_name'
# Drop the label 0 (corresponding to NaN value in orginial train_label csv)
# Reorder the labels to get athmospheric labels first, ground labels ('agriculture') after
label = label[(label!=0) & (label!='agriculture')]
label = np.insert(label, 0, 'image_name')
label = np.insert(label, 5, 'agriculture')

# Create a Pandas DataFrame with columns corresponding to labels
df_label = pd.DataFrame(columns=label)

# Fill the DataFrame df_label with image_name corresponding to orginial dataframe (df_na)
# Fill with 1 if the label is present in a given row
df_na = pd.read_csv(CSV_PATH, sep = ';')
for index in df_na.index:
    # Add new empty new row in df_label
    df_label = df_label.append(pd.Series(dtype='object'), ignore_index=True)
    
    # Get all the non-nan values of originial df_na
    index_values = df_na.loc[index].dropna().values
    
    # Add the image_name
    index_image_name = index_values[0]
    df_label.image_name[index] = index_image_name
    
    index_labels = index_values[1:]
    for labels in index_labels:
        df_label[labels][index] = 1   

df_label.head()

# Fill NaN values with 0
# Set image_name as index
df_label = df_label.fillna(0)
df_label.set_index(df_label.image_name, inplace = True)
df_label.drop(columns='image_name', inplace =True)

# Save DataFrame as csv file
NEW_PATH = './IPEO_Planet_project/new_train_labels.csv'
df_label.to_csv(path_or_buf=NEW_PATH, sep=',')

# PART 2

In [23]:
import os
import numpy as np
from skimage.io import imsave, imread
import matplotlib as plt
import pandas as pd

,haze,clear,cloudy,partly_cloudy,agriculture,conventional_mine,bare_ground,artisinal_mine,blooming,blow_down,cultivation,habitation,water,primary,road,selective_logging,slash_burn
0,2697,28431,2089,7261,12315,100,862,339,332,98,4477,3660,7411,37513,8071,340,209


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/Colab_Notebooks/Observation_Earth/IPEO-Understanding-the-Amazon-from-Space

In [ ]:
# Load the train_label
CSV_PATH = './CSV/new_train_labels.csv'
train_label = pd.read_csv(CSV_PATH)
label_sum = pd.DataFrame(columns = train_label.drop('image_name',axis=1).columns)
label_sum = label_sum.append(train_label.drop('image_name', axis=1).sum(), ignore_index = True)
label_sum

In [ ]:
rare_label_list = list()
for labels_name in label_sum.columns:
  if label_sum[labels_name].iloc[0] < 1000:
    rare_label_list.append([label_sum[labels_name].iloc[0], labels_name])
rare_label_list.sort()
rare_label_list = np.array(rare_label_list)
rare_label_list[:,1]

In [ ]:
all_index_rare = list(train_label[train_label[rare_label_list[:,1]].sum(axis=1)>0].index)
all_index_not_rare = list(set(train_label.index) - set(all_index_rare))
list_final = []

for rare in rare_label_list[:,1]:
  tempo_list = []
  for i in reversed(all_index_rare):
    if train_label[rare].loc[i] == 1:
      all_index_rare.remove(i)
      tempo_list.append(i)
  list_final.append(tempo_list)
list_final.append(all_index_not_rare)

In [ ]:
import pickle
with open('./List_label_rare/list_label_rare.npy', "wb") as fp:
  pickle.dump(list_final, fp)

# Part 3
Loading image as numpy matrix
- ! Don't rerun this part (+4h of loading, already exists)

import os
import numpy as np
from skimage.io import imsave, imread
import matplotlib as plt

from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Colab_Notebooks/Observation_Earth/IPEO-Understanding-the-Amazon-from-Space
#!git checkout Test_Louis
#!git pull

IMAGE_PATH = '../IPEO_Planet_project/train-jpg'

# Create numpy matrix representing image
numpy_image = np.ndarray(shape=(256,256,3,0))

for i in os.listdir(IMAGE_PATH):
  #obtain the index of the current image file (example : '0-999')
  index = i.split("_")[1]

  # Example : '../IPEO_Planet_project/train-jpg/train_0-999'
  train_tuple = os.path.join(IMAGE_PATH,i)

  for j in os.listdir(train_tuple):
    # Example : '../IPEO_Planet_project/train-jpg/train_0-999/train_1.jpg'
    image_directories = os.path.join(train_tuple, j)
    image_array = imread(image_directories)
    image_array = np.reshape(image_array, (256,256,3,1))
    numpy_image = np.concatenate((numpy_image, image_array), axis = 3)

  np.save(file = os.path.join(train_tuple, f"numpy_ndarray_{index}.npy"), arr = numpy_image)

  # Empty the ndarray to fill it with new image
  numpy_image = np.empty(shape=(256,256,3,0))